In [83]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import os
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from ipywidgets import fixed, Layout, Button, Box
import scipy.optimize as spopt 

%matplotlib widget
%matplotlib inline
files = [i for i in os.listdir('.') if '.txt' in i]

run_but = widgets.ToggleButton(
    value=False,
    description='Read File',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

dropdown = widgets.Dropdown(options=files, values=files[0],description='File name',disabled=False)



def get_bound(max_val,min_val):
    if max_val  < 0:
        boundary_max   = -.1 
    else:
        boundary_max   = .1
    if  min_val  < 0:
        boundary_min   = .1
    else:
        boundary_min   = -.1
        return min_val*(1+boundary_min),max_val*(1+boundary_max)

def monoExp(x,a,t):
    return a*np.exp(-x/t)
def trpl_plot(dataframe,bin_num,tot_len):
    plt.clf() 
    #style.use('dark_background')
    fluorescence = dataframe['Time'].to_numpy(float)
    hist, bins = np.histogram(fluorescence, bins=bin_num)
    
    #Plot generator:
    plt.rcParams['font.family'] = 'DeJavu Serif'
    plt.rcParams['font.serif'] = ['Times New Roman']
    
    plt.xlabel("Time (ns)", fontsize=18,labelpad=4)          #Defines the x axis' label parameters
    plt.ylabel("Intensity (a.u.)", fontsize=18,labelpad=4)   #Defines the y axis' label parameters
    plt.tick_params(axis='x', labelsize=15, pad=4)           #Customizes the x ticks (axis,size,vertical offset)  
    plt.ylim([-.05,1.05])                                    #Defines the y range
    plt.tick_params(axis='y', labelsize=15, pad=4)           #Customizes the y ticks (axis,size,horizontal offset)  
    
    initial_params=[1.0,10.0]
    popt, pcov = spopt.curve_fit(monoExp,bins[:-1],hist/max(hist))
    fit_curve = [monoExp(x,popt[0],popt[1]) for x in bins[:-1]]
    
    plt.plot(bins[:-1],hist/max(hist), color='red', label='Fluorescence')
    plt.plot(bins[:-1],fit_curve, color='blue', label='Fit')
    
    print('The fitted lifetime is:', "{:.2f}".format(popt[1]),'ns')
    print('We are using {:.2f} of the total data'.format(len(fluorescence)/tot_len))
    legx= plt.legend(bbox_to_anchor=(0.88,0.88), fontsize=12, frameon=False ) 
    plt.show()
def filter_data(data,mats,energy,particles,deaths):
    return data[( (data['Type'].isin(particles)) & (data['CausaMortis'].isin(deaths)) & \
                        (data.Location <= mats[1]) & (data.Location >= mats[0]) & \
                        (data.Energy <= energy[1]) & (data.Energy >= energy[0]) )]

def trpl(data,mats,energy,particles,deaths,tot_len):
    data_filtred = filter_data(data,mats,energy,particles,deaths)
    try:
        bin_slider = widgets.IntSlider(
        value=7,
        max=1E3,
        min=2,
        step=1,
        description='Number Bins:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
        )
        widgets.interact(trpl_plot,dataframe=fixed(data_filtred),bin_num = bin_slider,tot_len=fixed(tot_len))
    except:
        pass
    
def pre_load(file_name,run_button):
    if run_button:
        data = pd.read_csv(file_name, delim_whitespace=True)
        data = data[data.Time != 'END'] #removes the end of round lines
        #print('reading',file_name)
        tot_len = np.shape(data)[0]
        
        unique_mats   = data['Location'].unique()
        unique_death  = data['CausaMortis'].unique()
        unique_types  = data['Type'].unique()
        unique_energy = data['Energy'].unique()
        
        max_energy = np.amax(unique_energy)
        min_energy = np.amin(unique_energy)
        min_bound,max_bound = get_bound(max_energy,min_energy)

            
        #defining widgets after pre_load process
        energy = widgets.FloatRangeSlider(
            value=[min_energy, max_energy],
            min=min_bound,
            max=max_bound,
            step=(max_energy-min_energy)/4,
            description='Energy:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )        

        slider = widgets.IntRangeSlider(
            value=[np.amin(unique_mats), np.amax(unique_mats)],
            min=np.amin(unique_mats),
            max=np.amax(unique_mats),
            step=1,
            description='Materials:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
        )                
        deaths = widgets.SelectMultiple(
            options=unique_death,
            value=[unique_death[0]],
            #rows=10,
            description='Process',
            disabled=False,
        )
        particle_choose = widgets.SelectMultiple(
            options=unique_types,
            value=[unique_types[0]],
            #rows=10,
            description='Types',
            disabled=False,
        )
        w = widgets.interactive(trpl,data=fixed(data),mats = slider,energy=energy,particles=particle_choose,deaths= deaths,tot_len=fixed(tot_len))

        accordion = widgets.Accordion(children=[w], selected_index=None)

        accordion.set_title(0, 'TRPL')
        accordion.set_title(1, 'HISTOGRAM')
        accordion.set_title(2, 'HEATMAP')
        display(accordion)

    else:
        pass   
        

In [84]:
widgets.interact(pre_load, file_name=dropdown,run_button=run_but);


interactive(children=(Dropdown(description='File name', options=('Simulation_NPB_0.422_D_39.46_.txt', 'Simulat…